# LLMScope Demo Notebook

This notebooks illustrates a basic application of LLMScope to the ACI-Bench dataset.

Before using LLMScope, you may need to configure several environment variables, especially when working with local models. Some commonly used variables are described below:

* `LLMSCOPE_STORAGE_DIR` — Specifies the directory where LLMScope should store datasets, models, logs ad results. If not set, it defaults to a platform-specific application cache folder.
* `TOKENIZERS_PARALLELISM` — It is recommended to set this to `true` when using local models. However, if you encounter issues related to parallelism, you may need to set it to `false`.
* `CUDA_HOME` — May be required when using vLLM local models with NVIDIA GPUs. If not already defined in your environment, this should point to the root of your CUDA installation (e.g., `/usr/local/cuda-12.4.0`).
* `TORCH_CUDA_ARCH_LIST` — Helps suppress CUDA-related warnings when using local models with NVIDIA GPUs. You can determine the appropriate value(s) by running:
    ```
    nvidia-smi --query-gpu=compute_cap --format=csv
    ```

In [ ]:
%load_ext autoreload
%autoreload 2

import os

os.environ["LLMSCOPE_STORAGE_DIR"] = "/vol/bitbucket/ad5518/llmscope_cache"
os.environ["CUDA_HOME"] = "/vol/cuda/12.4.0/"
os.environ["TORCH_CUDA_ARCH_LIST"] = "8.0"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

## Dataset
As a first step, we need to define a dataset. In this case, we will use the built-in [ACI-Bench dataset](https://www.nature.com/articles/s41597-023-02487-3).

In [ ]:
from llmscope.datasets.managers import AciBenchDatasetManager

aci_dataset_manager = AciBenchDatasetManager(splits=["train"])

## Generation Steps
The next step is to define how the data should be used to generate model outputs, including the prompts involved. This is achieved using the Solvers abstraction provided by the Inspect AI library. A list of available solvers can be found in the [official documentation](https://inspect.aisi.org.uk/solvers.html#built-in-solvers).

Solvers can be composed into a sequence, allowing multiple steps to be chained together, as illustrated below. The full generation steps need to be assigned a unique name — this enables the comparison of results across different experiments when varying prompts or solver configurations.

In [ ]:
from inspect_ai.solver import generate, prompt_template, system_message

from llmscope.generation import GenerationSteps

system_prompt_template = "You are an expert clinical assistant specialising in the creation of medically accurate summaries from a dialogue between the doctor and patient."
user_prompt_template = """Your task is to generate a clinical note based on a conversation between a doctor and a patient. Use the following format for the clinical note:

1. **CHIEF COMPLAINT**: [Brief description of the main reason for the visit]
2. **HISTORY OF PRESENT ILLNESS**: [Summary of the patient's current health status and any changes since the last visit]
3. **REVIEW OF SYSTEMS**: [List of symptoms reported by the patient]
4. **PHYSICAL EXAMINATION**: [Findings from the physical examination]
5. **RESULTS**: [Relevant test results]
6. **ASSESSMENT AND PLAN**: [Doctor's assessment and plan for treatment or further testing]

**Conversation:**
{prompt}

**Note:**
"""

aci_generation = GenerationSteps(
    name="Structured",
    solver=[
        system_message(system_prompt_template),
        prompt_template(user_prompt_template),
        generate(),
    ],
)

## Task Spec
In addition to selecting the dataset to be used, it is also necessary to specify which columns will serve as inputs, targets (i.g., ground-truth references, if available) and sample IDs. Optionally, additional columns can be retained as metadata — for example, if they are useful for the result analysis.

Some datasets may support multiple tasks, in which case a preprocessing step may be needed to adapt the data for a specific task. In this case, no task-specific adjustments are needed, so we use the default preprocessor that returns the data unchanged.

In [ ]:
from inspect_ai.dataset import FieldSpec

from llmscope.tasks import DefaultTaskPreprocessor

aci_field_spec = FieldSpec(
    input="dialogue",
    target="note",
    id="id",
    metadata=[
        "dataset",
        "encounter_id",
        "doctor_name",
        "patient_gender",
        "patient_age",
        "patient_firstname",
        "patient_familyname",
        "cc",
        "2nd_complaints",
    ],
)
dialogue_task_preprocessor = DefaultTaskPreprocessor(name="Dialogue")

## Model Config
Next, we specify the model(s) to be used in our experiment. We can use any model and model provider supported by Inspect AI, configurable through model arguments and generation configuration. For an overview of supported models and the arguments, please refer to the related [Inspect AI documentation](https://inspect.aisi.org.uk/models.html). In this demo, we use two local vLLM models — [Llama 3.1 8B](https://huggingface.co/meta-llama/Llama-3.1-8B) and [Phi 4 Mini Instruct](https://huggingface.co/microsoft/Phi-4-mini-instruct).

In [ ]:
from inspect_ai.model import GenerateConfigArgs

from llmscope.constants import MODELS_PATH
from llmscope.generation import ModelConfig

llama_config = ModelConfig(
    "vllm/meta-llama/Llama-3.1-8B-Instruct",
    model_args={
        "download_dir": MODELS_PATH,
        "device": "2",
        "gpu_memory_utilization": 0.9,
        "max_model_len": 8192,
    },
    generation_args=GenerateConfigArgs(
        seed=42,
        temperature=0.7,
        top_p=0.95,
        max_connections=128,
    ),
)

phi_config = ModelConfig(
    "vllm/microsoft/Phi-4-mini-instruct",
    model_args={
        "download_dir": MODELS_PATH,
        "device": "2",
        "gpu_memory_utilization": 0.9,
        "max_model_len": 8192,
    },
    generation_args=GenerateConfigArgs(
        seed=42,
        temperature=0.7,
        top_p=0.95,
        max_connections=128,
    ),
)

## Evaluation
In this step, we define the evaluation metrics. Here, we simply use the BLUE and ROUGE metrics as implemented in LLMScope. Custom metrics are also supported, provided that they are implemented as Inspect AI-compatible scorers and metrics. For reference and inspiration, please check the [LLMScope BLUE implementation](https://github.com/nhsengland/llmscope/blob/main/llmscope/evaluation/evaluators/bleu.py) and the [relevant Inspect AI documentation](https://inspect.aisi.org.uk/scorers.html).

In [ ]:
from llmscope.evaluation.evaluators import bleu_evaluator
from llmscope.evaluation.evaluators import rouge_evaluator

## Workflow
To bring everything together, we configure the overall task and experiment pipeline. For more extensive and complex evaluations, multiple experiment batch configurations can be specified.

In [ ]:
from llmscope.evaluation import ExperimentBatchConfig, TaskConfig

aci_task_config = TaskConfig(
    dataset_manager=aci_dataset_manager,
    generation_steps=aci_generation,
    field_spec=aci_field_spec,
    task_preprocessor=dialogue_task_preprocessor,
)

experiment_config = ExperimentBatchConfig(
    tasks=[aci_task_config],
    model_configs=[llama_config, phi_config],
    evaluators=[bleu_evaluator, rouge_evaluator],
)

To store all the logs, outputs and results associated with our experiments, we also create a new `Project` object, which is passed to the evaluation pipeline. When initialised, the project automatically loads any previously stored outputs and evaluation results assciated with the given project name. This ensures that any completed tasks are not unnecessarily rerun unless explicitly required by setting `force_rerun=True` when calling the pipeline's `run` method.

In [ ]:
from llmscope.workflow import Pipeline, Project

aci_project = Project(name="ACI-Bench Evaluation")

aci_pipeline = Pipeline(
    experiments=experiment_config,
    project=aci_project,
)

In [ ]:
aci_pipeline.run()

## Result Analysis
The results associated with the project can be analysed using dedicated result analysers. In this example, we use a simple tabular result analyser, which summarises all the evaluation metrics in a structured table format.

In [ ]:
import polars as pl

from llmscope.workflow.analysers import TabularResultAnalyser

analyser = TabularResultAnalyser[pl.DataFrame](output_format="polars")

In [ ]:
analyser(aci_project)